<a href="https://colab.research.google.com/github/rounak777/Wastify/blob/master/garbage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/garbage project/garbage dataset/Garbage classification/Garbage classification")
os.getcwd()

'/content/gdrive/My Drive/Colab Notebooks/garbage project/garbage dataset/Garbage classification/Garbage classification'

In [4]:
import numpy as np
import cv2
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D
from keras.models  import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
import random,os,glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [5]:
dir_path ="/content/gdrive/My Drive/Colab Notebooks/garbage project/garbage dataset/Garbage classification/Garbage classification"

In [6]:
img_list = glob.glob(os.path.join(dir_path, '*/*.jpg'))

In [7]:
len(img_list)

2529

In [8]:
train=ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         validation_split=0.1,
                         rescale=1./255,
                         shear_range = 0.1,
                         zoom_range = 0.1,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,)

In [9]:
test=ImageDataGenerator(rescale=1/255,
                        validation_split=0.1)

In [10]:
train_generator=train.flow_from_directory(dir_path,
                                          target_size=(32,32),
                                          batch_size=32,
                                          class_mode='categorical',
                                          subset='training')

Found 2278 images belonging to 6 classes.


In [11]:
test_generator=test.flow_from_directory(dir_path,
                                        target_size=(32,32),
                                        batch_size=32,
                                        class_mode='categorical',
                                        subset='validation')

Found 251 images belonging to 6 classes.


In [12]:
labels = (train_generator.class_indices)
print(labels)

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [13]:
labels = dict((v,k) for k,v in labels.items())
print(labels)

{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [14]:
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((32, 32, 32, 3), (32, 6))

In [15]:
print (train_generator.class_indices)

Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(Labels)

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [16]:
model=Sequential()
#Convolution blocks

model.add(Conv2D(32,(3,3), padding='same',input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
#model.add(SpatialDropout2D(0.5)) # No accuracy

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 
#model.add(SpatialDropout2D(0.5))

model.add(Conv2D(32,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2)) 

#Classification layers
model.add(Flatten())

model.add(Dense(32,activation='relu'))
#model.add(SpatialDropout2D(0.5))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

filepath="trained_model.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 32)          18464     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)              

In [18]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc']) # RMS PROP - No accuracy

#es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [19]:
history = model.fit_generator(train_generator,
                              epochs=100,
                              steps_per_epoch=2276//32,
                              validation_data=test_generator,
                              validation_steps=251//32,
                              workers = 4,
                              callbacks=callbacks_list) 
#41 epoch - 75% #73- 76.9%
#78 epoch - 80%

Epoch 1/100
71/71 [==============================] - 12s 171ms/step - loss: 1.7472 - acc: 0.2235 - val_loss: 1.7102 - val_acc: 0.2277

Epoch 00001: val_acc improved from -inf to 0.22768, saving model to trained_model.h5
Epoch 2/100
71/71 [==============================] - 11s 152ms/step - loss: 1.7107 - acc: 0.2244 - val_loss: 1.5938 - val_acc: 0.2466

Epoch 00002: val_acc improved from 0.22768 to 0.24658, saving model to trained_model.h5
Epoch 3/100
71/71 [==============================] - 11s 152ms/step - loss: 1.6487 - acc: 0.2876 - val_loss: 1.6623 - val_acc: 0.2603

Epoch 00003: val_acc improved from 0.24658 to 0.26027, saving model to trained_model.h5
Epoch 4/100
71/71 [==============================] - 10s 148ms/step - loss: 1.5523 - acc: 0.3571 - val_loss: 1.5017 - val_acc: 0.2740

Epoch 00004: val_acc improved from 0.26027 to 0.27397, saving model to trained_model.h5
Epoch 5/100
71/71 [==============================] - 10s 148ms/step - loss: 1.4659 - acc: 0.3954 - val_loss: 1.

In [ ]:
from keras.preprocessing import image

img_path = '/content/gdrive/My Drive/Colab Notebooks/garbage project/garbage dataset/Garbage classification/Garbage classification/plastic/plastic75.jpg'


img = image.load_img(img_path, target_size=(300, 300))
img = image.img_to_array(img, dtype=np.uint8)
img=np.array(img)/255.0

plt.title("Loaded Image")
plt.axis('off')
plt.imshow(img.squeeze())

p=model.predict(img[np.newaxis, ...])

#print("Predicted shape",p.shape)
print("Maximum Probability: ",np.max(p[0], axis=-1))
predicted_class = labels[np.argmax(p[0], axis=-1)]
print("Classified:",predicted_class)

In [ ]:
classes=[]
prob=[]
print("\n-------------------Individual Probability--------------------------------\n")

for i,j in enumerate (p[0],0):
    print(labels[i].upper(),':',round(j*100,2),'%')
    classes.append(labels[i])
    prob.append(round(j*100,2))
    
def plot_bar_x():
    # this is for plotting purpose
    index = np.arange(len(classes))
    plt.bar(index, prob)
    plt.xlabel('Labels', fontsize=12)
    plt.ylabel('Probability', fontsize=12)
    plt.xticks(index, classes, fontsize=12, rotation=20)
    plt.title('Probability for loaded image')
    plt.show()
plot_bar_x()

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

# ________________ Graph 1 -------------------------

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

# ________________ Graph 2 -------------------------

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import tensorflow as tf
import keras

In [ ]:
keras.models.save_model(model,'final_model.hdf5')

In [ ]:
from IPython.display import FileLinks
FileLinks('.')